In [171]:
from binance import Client
client = Client(api_key="VQaW84v81fvM6b0z2cG97q0FBGK0VDVNYZGxPlMYkL1caao8u7lSrSWxeE5c6NKq",api_secret="9XgnO5nds9V8UVw5O8D0J6NHNPqL4ojj5t3FxZs7xqc8sd8MwEt44QshkT3W9auM",tld='com')

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd
from datetime import datetime, timedelta
from time import sleep

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    # print('Indicadores agregados')

def señal_MACD(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0:
            buy.append(i)
        elif df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0:
            sell.append(i)
    return buy, sell

In [212]:
day = datetime.utcnow() - timedelta(1)
PAIR = 'BTCTUSD'
LOCALIDAD_SEÑAL_ENTRADA = 10


with connection.connect() as conn:
    df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
df = df.sort_values(by='Tiempo')
df

,Precio
Tiempo,
2023-07-04 22:30:29.271,30789.22
2023-07-04 22:30:29.274,30788.41
2023-07-04 22:30:31.433,30788.39
2023-07-04 22:30:32.216,30788.57
2023-07-04 22:30:32.859,30789.17
...,...
2023-07-05 14:39:57.878,30409.96
2023-07-05 14:40:00.774,30411.68
2023-07-05 14:40:03.797,30410.97


In [210]:
tiempo = pd.to_datetime('2023-07-05 02:09:35.627')

while True:
    with connection.connect() as conn:
        df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
    df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
    df = df.sort_values(by='Tiempo')
            
    # MACD(df)
    # buy, sell = señal_MACD(df)

    # buy = df.Precio.iloc[buy]
    # sell = df.Precio.iloc[sell]
    # DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buyprices.index[-1]).total_seconds()

    if tiempo != df.index[-1]:
        print('tasa de actualización: ', (df.index[-1] - tiempo).total_seconds(), 'tiempo:', datetime.utcnow())
        tiempo = df.index[-1]

    if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA:
        print('Señal')


    

tasa de actualización:  40147.0 tiempo: 2023-07-05 13:18:47.648183
tasa de actualización:  7.191 tiempo: 2023-07-05 13:18:53.653132
tasa de actualización:  6.4 tiempo: 2023-07-05 13:19:01.440571
tasa de actualización:  8.563 tiempo: 2023-07-05 13:19:08.687455
tasa de actualización:  6.833 tiempo: 2023-07-05 13:19:16.072360
tasa de actualización:  7.326 tiempo: 2023-07-05 13:19:23.288623
tasa de actualización:  8.042 tiempo: 2023-07-05 13:19:30.754081
tasa de actualización:  7.088 tiempo: 2023-07-05 13:19:38.103876
tasa de actualización:  6.674 tiempo: 2023-07-05 13:19:45.417192
tasa de actualización:  8.328 tiempo: 2023-07-05 13:19:52.844142
tasa de actualización:  7.057 tiempo: 2023-07-05 13:20:00.043868
tasa de actualización:  6.9559999999999995 tiempo: 2023-07-05 13:20:07.420476
tasa de actualización:  8.111 tiempo: 2023-07-05 13:20:14.857796
tasa de actualización:  6.914 tiempo: 2023-07-05 13:20:22.076677
tasa de actualización:  7.128 tiempo: 2023-07-05 13:20:29.247435
tasa de actu

KeyboardInterrupt: 

In [39]:
buy[-50:-1]

Tiempo
2023-07-05 00:06:30.799    30807.35
2023-07-05 00:07:05.620    30807.69
2023-07-05 00:07:14.598    30806.81
2023-07-05 00:07:24.433    30808.34
2023-07-05 00:07:45.125    30808.11
2023-07-05 00:08:05.642    30810.00
2023-07-05 00:08:15.804    30810.34
2023-07-05 00:08:30.902    30807.62
2023-07-05 00:08:34.971    30807.53
2023-07-05 00:09:10.490    30808.21
2023-07-05 00:09:25.810    30809.04
2023-07-05 00:09:35.417    30809.01
2023-07-05 00:09:50.990    30808.77
2023-07-05 00:10:18.595    30811.40
2023-07-05 00:10:37.610    30810.32
2023-07-05 00:11:11.607    30789.77
2023-07-05 00:12:12.014    30789.28
2023-07-05 00:12:16.177    30789.43
2023-07-05 00:12:25.190    30789.58
2023-07-05 00:12:53.790    30799.01
2023-07-05 00:13:29.075    30800.44
2023-07-05 00:13:32.145    30800.30
2023-07-05 00:13:34.519    30801.13
2023-07-05 00:13:48.925    30801.75
2023-07-05 00:14:18.670    30799.21
2023-07-05 00:14:41.660    30795.00
2023-07-05 00:15:01.752    30787.61
2023-07-05 00:16:04.3

In [28]:
sellprices = pd.Series(dtype=float)
buyprices = pd.Series(dtype=float)

while True:
    if sell.index[0] < buy.index[0]:
        sell = sell.drop(sell.index[0])
    elif buy.index[-1] > sell.index[-1]:
        buy = buy.drop(buy.index[-1])

    if buy.index[0] < sell.index[0] and buy.index[-1] < sell.index[-1]:
        break

p = 0
while True:

    if len(buyprices) == 0:
        buyprices = pd.concat([buyprices,buy[0:1]])
        sellprices = pd.concat([sellprices,sell[0:1]])
        j = 0
        continue

    if len(buy[buy.index > buyprices.index[j]]) == p:
                break

    if len(buyprices) > 0:
        p = 0
        x = len(buyprices)
        while True:    
            if x != len(buyprices):
                x = len(buyprices)
                p = 0

            if buy[buy.index > buyprices.index[j]].index[p] > sellprices.index[j]:
                buyprices = pd.concat([buyprices,buy[buy.index > buyprices.index[j]][p:p+1]])
                k = 0
                while True:
                    if buyprices.index[j+1] < sell[sell.index > sellprices.index[j]].index[k]:
                        sellprices = pd.concat([sellprices,sell[sell.index > sellprices.index[j]][k:k+1]])
                        j += 1
                        break
                    k += 1
            p += 1

            if len(buy[buy.index > buyprices.index[j]]) == p:
                break

In [29]:
len(buy)

113

In [30]:
len(sellprices)

112

In [31]:
profits = []

for i in range(len(buyprices)):
    profit = buyprices[i] - sellprices[i]
    profits.append(profit*0.005)
profits

[0.0031000000000130966,
 0.005499999999992724,
 0.008999999999996362,
 0.002749999999996362,
 0.04335000000000946,
 0.004799999999995634,
 0.004400000000005093,
 0.011550000000006548,
 0.015599999999994908,
 0.008500000000003638,
 0.0061999999999898135,
 0.012749999999996361,
 0.009600000000009459,
 0.006350000000002183,
 0.01890000000001237,
 -0.059149999999990543,
 0.009950000000008005,
 0.005200000000004366,
 0.0025,
 0.008099999999994906,
 0.00930000000000291,
 -0.0075,
 0.004050000000006548,
 0.0035499999999956343,
 0.004300000000002911,
 0.019300000000002912,
 0.0045499999999992725,
 0.008099999999994906,
 -0.005599999999994907,
 0.006149999999997817,
 0.008250000000007276,
 0.004650000000001455,
 0.002649999999994179,
 0.006650000000008731,
 0.0075,
 0.0041499999999905415,
 0.00444999999999709,
 0.012549999999991996,
 0.0020499999999992725,
 0.004650000000001455,
 0.0018500000000130968,
 0.004099999999998545,
 -0.02390000000001237,
 -0.011749999999992725,
 0.01875,
 -0.010249999

In [32]:
sum(profits)

0.6250999999999477